# (1) 금리 데이터 merge

In [ ]:
data = pd.read_csv('/Users/parkhyuniee/Desktop/IM/project_data/capital_data.csv')

In [ ]:
rate_data = pd.read_csv('/Users/parkhyuniee/Desktop/IM/project_data/한국은행 기준금리 및 여수신금리_06105056.csv')

# 거래년월을 datetime으로 변환하고, 연도와 분기 추출
data['거래년월'] = pd.to_datetime(data['거래년월'])
data['연도'] = data['거래년월'].dt.year
data['분기'] = data['거래년월'].dt.quarter

# 연도와 분기를 기준으로 매칭하기 위해 새로운 열 '연도/분기' 생성
data['연도_분기'] = data['연도'].astype(str) + '/Q' + data['분기'].astype(str)

# 금리 데이터에서도 같은 형식으로 '연도/분기' 열을 생성
rate_data = rate_data.melt(id_vars=['통계표', '계정항목', '단위', '변환'], var_name='연도_분기', value_name='금리')

# 필요한 금리 데이터만 필터링 (예: '한국은행 기준금리')
target_rate_data = rate_data[rate_data['계정항목'] == '한국은행 기준금리'][['연도_분기', '금리']]

# 이미지 데이터와 금리 데이터를 '연도/분기'를 기준으로 병합
merged_data = pd.merge(data, target_rate_data, on='연도_분기', how='left')

# 결과 출력
merged_data.head()

In [ ]:
# 연도_분기에서 '/' 제거
merged_data['연도_분기'] = merged_data['연도_분기'].str.replace('/', '')

In [ ]:
merged_data.to_csv('/Users/parkhyuniee/Desktop/IM/project_data/IR_merged_data.csv', index=False)

# (2) 소비자심리지수 merge

In [ ]:
cus_psy_data = pd.read_csv('/Users/parkhyuniee/Desktop/IM/project_data/cus_psy_data.csv')
IR_merged_data =  pd.read_csv('/Users/parkhyuniee/Desktop/IM/project_data/IR_merged_data.csv')

In [ ]:
cus_psy_merged_data = pd.merge(IR_merged_data, cus_psy_data, on='연도_분기', how='left')

# (3) 인구 데이터 merge

In [ ]:
population_data =pd.read_csv('/Users/parkhyuniee/Desktop/IM/project_data/population_data.csv')

### 1. 자택인구 기준으로 붙이기 

In [ ]:
cus_psy_merged_data['자택_시도'] = cus_psy_merged_data['자택_시도'].astype(str)
cus_psy_merged_data['자택_시군구'] = cus_psy_merged_data['자택_시군구'].astype(str)
cus_psy_merged_data['년도'] = cus_psy_merged_data['년도'].astype(str)

population_data['자택_시도'] = population_data['자택_시도'].astype(str)
population_data['자택_시군구'] = population_data['자택_시군구'].astype(str)
population_data['년도'] = population_data['년도'].astype(str)

# 자택_시도, 자택_시군구, 년도를 기준으로 병합
final_data = pd.merge(
    cus_psy_merged_data,
    population_data,
    on=['자택_시도', '자택_시군구', '년도'],  # 병합 기준 열
    how='left'                                # left join 사용
)

# 결과 확인
final_data.head()

In [ ]:
# column name 변경
population_data.rename(columns={'가맹점_광역시도': '자택_시도'},inplace=True)
population_data.rename(columns={'가맹점_시군구': '자택_시군구'},inplace=True)

### 2. 가맹점 인구 기준으로 붙이기

In [ ]:
final_data['가맹점_광역시도'] = final_data['가맹점_광역시도'].astype(str)
final_data['가맹점_시군구'] = final_data['가맹점_시군구'].astype(str)
final_data['년도'] = final_data['년도'].astype(str)

population_data['가맹점_광역시도'] = population_data['가맹점_광역시도'].astype(str)
population_data['가맹점_시군구'] = population_data['가맹점_시군구'].astype(str)
population_data['년도'] = population_data['년도'].astype(str)

# 자택_시도, 자택_시군구, 년도를 기준으로 병합
r_final_data = pd.merge(
    final_data,
    population_data,
    on=['가맹점_광역시도', '가맹점_시군구', '년도'],  # 병합 기준 열
    how='left'                                # left join 사용
)

# 결과 확인
r_final_data.head()

In [ ]:
r_final_data = r_final_data.rename(columns={'인구_y': '가맹점_인구'})
r_final_data = r_final_data.rename(columns={'인구_x': '자택_인구'})

In [ ]:
r_final_data.to_csv('/Users/parkhyuniee/Desktop/IM/project_data/final_data.csv', index = False)

# (4) 지역에서 '시'까지만 남기고 '구' 제거


In [ ]:
# 전체 데이터에서 제거
r_final_data['가맹점_시군구'] = r_final_data['가맹점_시군구'].str.replace(r'시\s.*구', '시', regex=True)
r_final_data['자택_시군구'] = r_final_data['자택_시군구'].str.replace(r'시\s.*구', '시', regex=True)

In [ ]:
# 인구 데이터에서 제거 
population_data['가맹점_시군구'] = population_data['가맹점_시군구'].str.replace(r'시\s.*구', '시', regex=True)